In [1]:
import pandas as pd
import csv
from sqlalchemy import create_engine
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import numpy as np


In [4]:
file_one = "Data/April_Motorvehicles.csv" 

crashes = pd.read_csv(file_one, encoding="ISO-8859-1") 

crashes.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,04/01/2020,21:23,BROOKLYN,11219.0,40.634270,-73.983310,"(40.63427, -73.98331)",NaN,NaN,1575 44 STREET,...,Unspecified,NaN,NaN,NaN,4308629,Station Wagon/Sport Utility Vehicle,Van,NaN,NaN,NaN
1,04/01/2020,16:09,NaN,NaN,40.665660,-73.767624,"(40.66566, -73.767624)",FARMERS BOULEVARD,145 ROAD,NaN,...,Unspecified,NaN,NaN,NaN,4305069,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,04/01/2020,21:18,BROOKLYN,11249.0,40.713078,-73.964080,"(40.713078, -73.96408)",BERRY STREET,SOUTH 3 STREET,NaN,...,Failure to Yield Right-of-Way,NaN,NaN,NaN,4307292,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN
3,04/01/2020,14:29,NaN,NaN,40.627926,-73.931750,"(40.627926, -73.93175)",AVENUE J,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4305138,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,04/01/2020,17:00,QUEENS,11378.0,40.725930,-73.918370,"(40.72593, -73.91837)",56 ROAD,50 STREET,NaN,...,Unspecified,NaN,NaN,NaN,4305053,Sedan,Box Truck,NaN,NaN,NaN


In [6]:
del crashes['ON STREET NAME']
del crashes['CROSS STREET NAME']
del crashes['OFF STREET NAME']
del crashes['CONTRIBUTING FACTOR VEHICLE 3']
del crashes['CONTRIBUTING FACTOR VEHICLE 4']
del crashes['CONTRIBUTING FACTOR VEHICLE 5']
del crashes['VEHICLE TYPE CODE 3']
del crashes['VEHICLE TYPE CODE 4']
del crashes['VEHICLE TYPE CODE 5']

In [7]:
crashes

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,04/01/2020,21:23,BROOKLYN,11219.0,40.634270,-73.983310,"(40.63427, -73.98331)",0,0,0,0,0,0,0,0,Unspecified,Unspecified,4308629,Station Wagon/Sport Utility Vehicle,Van
1,04/01/2020,16:09,NaN,NaN,40.665660,-73.767624,"(40.66566, -73.767624)",0,0,0,0,0,0,0,0,Unspecified,Unspecified,4305069,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle
2,04/01/2020,21:18,BROOKLYN,11249.0,40.713078,-73.964080,"(40.713078, -73.96408)",1,0,0,0,1,0,0,0,Failure to Yield Right-of-Way,Failure to Yield Right-of-Way,4307292,Station Wagon/Sport Utility Vehicle,E-Bike
3,04/01/2020,14:29,NaN,NaN,40.627926,-73.931750,"(40.627926, -73.93175)",0,0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unspecified,4305138,Sedan,Station Wagon/Sport Utility Vehicle
4,04/01/2020,17:00,QUEENS,11378.0,40.725930,-73.918370,"(40.72593, -73.91837)",0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4305053,Sedan,Box Truck
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,04/30/2020,5:05,NaN,NaN,40.738842,-73.814040,"(40.738842, -73.81404)",0,0,0,0,0,0,0,0,Pavement Slippery,NaN,4311104,Pick-up Truck,NaN
4117,04/30/2020,15:15,NaN,NaN,40.839478,-73.883896,"(40.839478, -73.883896)",0,0,0,0,0,0,0,0,Oversized Vehicle,Unspecified,4311174,Tractor Truck Diesel,Sedan
4118,04/30/2020,21:30,NaN,NaN,40.716652,-73.825900,"(40.716652, -73.8259)",0,0,0,0,0,0,0,0,Unspecified,Unspecified,4311158,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle
4119,04/30/2020,16:00,BROOKLYN,11233.0,40.669600,-73.919690,"(40.6696, -73.91969)",0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4311361,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle


In [11]:
crashes = crashes[crashes['BOROUGH'].notna()]
crashes = crashes[crashes['LATITUDE'].notna()]
crashes = crashes[crashes['LONGITUDE'].notna()]



In [13]:
factor_df = crashes.groupby(['CONTRIBUTING FACTOR VEHICLE 1']).count()



In [21]:
factor_df = factor_df.reset_index()

In [26]:
factor_df = factor_df[['CONTRIBUTING FACTOR VEHICLE 1', 'COLLISION_ID']]

In [28]:
factordf = factor_df.rename(columns={"CONTRIBUTING FACTOR VEHICLE 1":"factor", "COLLISION_ID": "crashes"})

In [29]:
factordf

,factor,crashes
0,Accelerator Defective,2
1,Aggressive Driving/Road Rage,31
2,Alcohol Involvement,46
3,Animals Action,8
4,Backing Unsafely,121
5,Brakes Defective,11
6,Cell Phone (hand-Held),2
7,Cell Phone (hands-free),1
8,Driver Inattention/Distraction,583
9,Driver Inexperience,40


In [33]:
factordf = factordf.sort_values(by=['crashes'], ascending=False)


In [34]:
factordf

,factor,crashes
40,Unspecified,827
8,Driver Inattention/Distraction,583
13,Failure to Yield Right-of-Way,148
4,Backing Unsafely,121
36,Traffic Control Disregarded,92
39,Unsafe Speed,80
27,Passing or Lane Usage Improper,80
22,Other Vehicular,73
26,Passing Too Closely,73
16,Following Too Closely,60


In [35]:
factordf.to_csv("Data/Factors.csv", index=False, header=True)

